In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
#Importing relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression  #Subclass Linear Regression in Class Linear Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import datetime
import plotly.graph_objects as go
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# df=pd.read_csv("/kaggle/input/brent-oil-prices/BrentOilPrices.csv")
c


In [8]:
link = 'https://drive.google.com/file/d/1Qn3wgZOMxS5_lb0yaNfU7F-a9gw5ayw2/view?usp=sharing' # The shareable link
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('BrentOilPrices.csv')  
df = pd.read_csv('BrentOilPrices.csv')
df.describe(include = "all")
# Dataset is now stored in a Pandas Dataframe

sharing
sharing


ApiRequestError: ignored

In [ ]:
df=df.reset_index()
df=df.dropna()
print(df.head())
print(df.tail())

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df['Date']=pd.to_datetime(df["Date"])
df.dtypes

In [ ]:
#Preprocessing Data
#Function to prepare training, testing and forecasting data
#Function arguments are dataframe, column to be forecasted, no. of days to be predicted in future,test data size
def prepare_data(df,forecast_col,forecast_out,test_size):
    label = df[forecast_col].shift(-forecast_out) #creating the label column and 
    #expanding the dataset by a given number of rows having value NaN(not a number)
    #print(f"Length of the Label is /n {label[15:]}")
    print(label[-15:])
    X = np.array(df[[forecast_col]]) #creating the feature array
    X = preprocessing.scale(X) #preprocessing the feature array to reduce the biasness among input features
    X_lately = X[-forecast_out:] #dropping the last n rows which are newly created for forecasting
    # and X_lately column will be used later in predicting method
    X = X[:-forecast_out] #training and test data
    label.dropna(inplace = True) #dropping the NA values
    y = np.array(label) #assigning the output y
    
    #Creating training and test data
    x_train = X[0:int(len(X)*(1-test_size))]
    x_test = X[-int(len(X)*test_size):]
    y_train = y[0:int(len(y)*(1-test_size))]
    y_test = y[-int(len(y)*test_size):]
    response = [x_train,x_test,y_train,y_test,X_lately]
    #list of lists : list comprising of training i/p, test set, training o/p, test o/p, final 10 days that we need to predict as forecasting values
    return response

In [ ]:
forecast_col = 'Price' #column to be forecasted
forecast_out = 10 # how far to forecast i.e forecasting for 10 days
test_size = 0.2 # size of test data
x_train,x_test,y_train,y_test,X_lately = prepare_data(df,forecast_col,forecast_out,test_size)



In [ ]:
def train_model(model_class,x_train,y_train,x_test,y_test,X_lately):
    learner = model_class #Algorithm we want to initialise
    learner.fit(x_train,y_train) #learning happens here when the model is trained (training input and output variables)
    #y = f(x), function in the LR equation is given by learner.fit(x_train,y_train)
    score = learner.score(x_test,y_test) #testing the model
    
    #Prediction: We'll predict for a given set of dates
    #Forecast: We'll forecast for dates which are in Future
    
    forecast = learner.predict(X_lately) #set that'll contain forcasted data
    
    response = {} #creating json object/dictionary
    response['test_score'] = score #first key is test_score
    #measures the squared error b/w actual value and predicted value and provides a score b/w 0 and 1
    response['forecast_set'] = forecast #passing the forecasted values
    print(response)
    
    return learner

In [ ]:
learner = train_model(model_class = LinearRegression(),x_train = x_train,y_train = y_train
                      ,x_test = x_test,y_test = y_test,X_lately = X_lately)

In [ ]:
learner.fit_intercept,learner.intercept_ ,learner.coef_


In [ ]:
# Y=mX+C
#m=32.08
#c=46.477
#Y=32.08+46.47X

In [ ]:
#Plotting the interactive graphs#

In [ ]:
fig = go.Figure()
#add_trace: plotting line graph 
#Scatter: plots points b/w 2 points where y is in date format
#plotting training data represented by teal
fig.add_trace(go.Scatter(x = df["Date"], y = df["Price"],
                        mode = 'lines',
                        name = 'Price', line = dict(color='teal')))
#plotting prediction of date represented by blue
#green colored is actual value and blue color is predicted value
fig.add_trace(go.Scatter(x = df["Date"][-len(x_test):], y = learner.predict(x_test),
                        mode = 'lines',
                        name = 'Predicted Price', line = dict(color='blue')))
#plotting forecast data represented by red (future forecast of 10 days)
fig.add_trace(go.Scatter(x = pd.date_range('2022-05-15','2022-05-24',
                                           freq = pd.tseries.offsets.BDay()),
                        y = learner.predict(X_lately),
                        mode = 'lines',
                        name = 'Forecasted Price', line = dict(color='yellow')))
#DateOffset: Standard kind of date increment used for a date range
#BDay: DateOffset subclass representing possibly n business days
fig.update_layout(title = 'Brent Oil Prediction & Forecasting over 10 days',
                 plot_bgcolor = 'black',
                 xaxis = dict(showgrid = False), yaxis = dict(showgrid = False),
                 xaxis_title = "Date",
                 yaxis_title = "Brent Oil Prices ($)")

fig.show()
